In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("dataset/steam_cleaned_cluster.csv")
data

,Unnamed: 0.1,Unnamed: 0,name,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,...,categories_Local Co-op,categories_MMO,categories_VR Support,categories_Mods,categories_Mods (require HL2),categories_Steam Turn Notifications,age,Top10,appid,cluster
0,0,10,Counter-Strike: Source,Action,147,76640,3497,6842,400,15000000,...,0,0,0,0,0,0,7000,"['action', 'fps', 'multiplayer', 'shooter', 't...",10,cluster_1
1,1,20,Left 4 Dead,Action,73,17951,948,897,278,7500000,...,0,0,0,0,0,0,5523,"['action', 'fps', 'multiplayer', 'shooter', 'c...",20,cluster_1
2,2,30,Darwinia,Indie;Strategy,0,472,158,182,273,750000,...,0,0,0,0,0,0,6745,"['fps', 'multiplayer', 'action', 'shooter', 'w...",30,cluster_1
3,3,40,Arx Fatalis,RPG,0,671,106,11,21,150000,...,0,0,0,0,0,0,6117,"['action', 'fps', 'multiplayer', 'shooter', 'a...",40,cluster_5
4,4,50,DOOM II,Action,0,2517,182,90,78,750000,...,0,0,0,0,0,0,5995,"['fps', 'action', 'singleplayer', 'shooter', '...",50,cluster_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,597,26800,Brood,Adventure;Casual;Indie;Early Access,0,6,1,0,0,10000,...,0,0,0,0,0,0,1735,"['appid', '2d', 'singleplayer', 'great_soundtr...",26800,cluster_5
598,598,26900,Helltower,Action;Adventure;Indie,12,2,0,0,0,10000,...,0,0,0,0,0,0,1742,"['appid', 'physics', 'singleplayer', '2d', 'sa...",26900,cluster_2
599,599,27000,Pixel Art Monster - Color by Number,Action;Adventure;Casual;Indie;RPG;Simulation;S...,0,3,0,0,0,10000,...,0,0,0,0,0,0,1734,"['appid', 'horror', 'exploration', 'adventure'...",27000,cluster_5
600,600,27020,Erannorth Reborn,RPG;Early Access,25,6,2,0,0,10000,...,0,0,0,0,0,0,1722,"['appid', 'atmospheric', 'simulation', 'single...",27020,cluster_3


In [2]:
data = data.drop(columns=["Unnamed: 0"])
data_num = data.select_dtypes(include=np.number)
data_num["owners"] = np.log10(data_num["owners"])
data_num

,Unnamed: 0.1,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,categories_Multi-player,categories_Online Multi-Player,...,categories_Online Co-op,categories_Shared/Split Screen,categories_Local Co-op,categories_MMO,categories_VR Support,categories_Mods,categories_Mods (require HL2),categories_Steam Turn Notifications,age,appid
0,0,147,76640,3497,6842,400,7.176091,7.19,1,0,...,0,0,0,0,0,0,0,0,7000,10
1,1,73,17951,948,897,278,6.875061,7.19,1,0,...,0,0,0,0,0,0,0,0,5523,20
2,2,0,472,158,182,273,5.875061,7.19,0,0,...,0,0,0,0,0,0,0,0,6745,30
3,3,0,671,106,11,21,5.176091,2.99,0,0,...,0,0,0,0,0,0,0,0,6117,40
4,4,0,2517,182,90,78,5.875061,3.99,1,0,...,0,0,0,0,0,0,0,0,5995,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,597,0,6,1,0,0,4.000000,3.99,0,0,...,0,0,0,0,0,0,0,0,1735,26800
598,598,12,2,0,0,0,4.000000,3.99,0,0,...,0,0,0,0,0,0,0,0,1742,26900
599,599,0,3,0,0,0,4.000000,2.89,0,0,...,0,0,0,0,0,0,0,0,1734,27000
600,600,25,6,2,0,0,4.000000,8.29,0,0,...,0,0,0,0,0,0,0,0,1722,27020


In [3]:
from sklearn.model_selection import train_test_split

data_x = data_num.drop(columns="average_playtime")
data_y = data_num[["average_playtime"]]
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, random_state=1234567)
print("Train input shape:", x_train.shape)
print("Test input shape:", x_test.shape)

Train input shape: (451, 38)
Test input shape: (151, 38)


In [5]:
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)
y_train_arr = y_train_scaled.ravel()
y_test_arr = y_test_scaled.ravel()

def evaluate(name, model, perm=20):
    print("===", name, "===")
    score = permutation_test_score(model, x_train_scaled, y_train_arr, n_permutations=perm)
    print(f"Score: {score[0]:.2f} (p-value {score[2]:.3f})")
    print(f"               Best: {1/(perm+1):.3f}")
    print()

print()

evaluate("Dummy Regressor (Baseline)", DummyRegressor())
evaluate("Linear Regression", LinearRegression())
evaluate("Decision Tree", DecisionTreeRegressor(random_state=0))
evaluate("Random Forest", RandomForestRegressor(random_state=0, max_samples=0.5), perm=10)
evaluate("Boosting Regressor", HistGradientBoostingRegressor(random_state=0), perm=5)
evaluate("Multi-layer Perceptron", MLPRegressor(random_state=0, solver="adam",hidden_layer_sizes=[100], max_iter=2000), perm=3)


=== Dummy Regressor (Baseline) ===
Score: -0.16 (p-value 0.667)
               Best: 0.048

=== Linear Regression ===
Score: -2.12 (p-value 0.810)
               Best: 0.048

=== Decision Tree ===
Score: -0.22 (p-value 0.048)
               Best: 0.048

=== Random Forest ===
Score: 0.49 (p-value 0.091)
               Best: 0.091

=== Boosting Regressor ===
Score: -1.92 (p-value 1.000)
               Best: 0.167

=== Multi-layer Perceptron ===
Score: -0.10 (p-value 0.250)
               Best: 0.250



In [6]:
from sklearn.model_selection import GridSearchCV
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import r2_score

parameter_space = {
    "hidden_layer_sizes": [(50,), (50,50), (25,), (25,25,)], # Have some smaller sizes as we overfitted earlier.
    "activation": ["tanh", "relu"],
    "solver": ["sgd", "adam"],
    "alpha": [0.0001, 0.05],
    "learning_rate": ["constant","adaptive"],
}

# Ignore ConvergenceWarning to avoid spamming the output.
# SGD does not converge with 100 iterations in this dataset.
warnings.filterwarnings(action="ignore", category=ConvergenceWarning)

mlp = MLPRegressor(random_state=0, max_iter=100)
gridSearch = GridSearchCV(mlp, parameter_space, cv=5)
gridSearch.fit(x_train_scaled, y_train_arr)

train_pred = gridSearch.predict(x_train_scaled)
test_pred = gridSearch.predict(x_test_scaled)
print(f"The best parameters were:\n{gridSearch.best_params_}")
print("Score: ", r2_score(y_train_scaled, train_pred))

The best parameters were:
{'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (50, 50), 'learning_rate': 'constant', 'solver': 'adam'}
Score:  0.9968508041019319


In [7]:
warnings.resetwarnings()
mlp = MLPRegressor(random_state=0, activation="relu", solver="sgd", learning_rate="constant", alpha=0.05, hidden_layer_sizes=(50,50), max_iter=1000)
evaluate("MLP Fine Tuned", mlp, perm=2)

=== MLP Fine Tuned ===


C:\Users\DELL G15\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\DELL G15\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\DELL G15\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\DELL G15\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reac

Score: -0.09 (p-value 0.333)
               Best: 0.333



C:\Users\DELL G15\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
